<a href="https://colab.research.google.com/github/CatarinaL/ddosLogs/blob/master/ddos_logs_logit_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
!pip install category_encoders

import numpy as np
import pandas as pd
from scipy.io import arff
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import category_encoders as ce
from category_encoders.wrapper import PolynomialWrapper

In [35]:
#/content/drive/MyDrive/datasets/ddosLogs/final-dataset.arff
np.random.seed(123)
file = "/content/drive/MyDrive/datasets/ddosLogs/final-dataset.arff"

data, meta = arff.loadarff(file)
df = pd.DataFrame(data=data)
df.info

<bound method DataFrame.info of          SRC_ADD  DES_ADD  ...  LAST_PKT_RESEVED     PKT_CLASS
0           3.00    24.30  ...         50.021920     b'Normal'
1          15.00    24.15  ...         50.030211     b'Normal'
2          24.15    15.00  ...         50.060221  b'UDP-Flood'
3          24.90     9.00  ...         50.060098     b'Normal'
4          24.80     8.00  ...         50.061864     b'Normal'
...          ...      ...  ...               ...           ...
2160663    24.10    10.00  ...         50.040562     b'Normal'
2160664    24.12    12.00  ...         50.051067     b'Normal'
2160665     1.00    24.10  ...         50.013418     b'Normal'
2160666     5.00    24.50  ...         50.027339     b'Normal'
2160667    15.00    24.15  ...         50.030211     b'Normal'

[2160668 rows x 28 columns]>

In [36]:
df

,SRC_ADD,DES_ADD,PKT_ID,FROM_NODE,TO_NODE,PKT_TYPE,PKT_SIZE,FLAGS,FID,SEQ_NUMBER,NUMBER_OF_PKT,NUMBER_OF_BYTE,NODE_NAME_FROM,NODE_NAME_TO,PKT_IN,PKT_OUT,PKT_R,PKT_DELAY_NODE,PKT_RATE,BYTE_RATE,PKT_AVG_SIZE,UTILIZATION,PKT_DELAY,PKT_SEND_TIME,PKT_RESEVED_TIME,FIRST_PKT_SENT,LAST_PKT_RESEVED,PKT_CLASS
0,3.00,24.30,389693.0,21.0,23.0,b'tcp',1540.0,b'-------',4.0,11339.0,16091.0,24780100.0,b'Switch1',b'Router',35.529786,35.529786,35.539909,0.0,328.240918,505490.0,1540.0,0.236321,0.0,35.519662,35.550032,1.000000,50.021920,b'Normal'
1,15.00,24.15,201196.0,23.0,24.0,b'tcp',1540.0,b'-------',16.0,6274.0,16092.0,24781700.0,b'Router',b'server1',20.176725,20.176725,20.186848,0.0,328.205808,505437.0,1540.0,0.236337,0.0,20.156478,20.186848,1.000000,50.030211,b'Normal'
2,24.15,15.00,61905.0,23.0,22.0,b'ack',55.0,b'-------',16.0,1930.0,16092.0,885060.0,b'Router',b'Switch2',7.049955,7.049955,7.059958,0.0,328.206042,18051.3,55.0,0.008441,0.0,7.039952,7.069962,1.030045,50.060221,b'UDP-Flood'
3,24.90,9.00,443135.0,23.0,21.0,b'ack',55.0,b'-------',10.0,12670.0,16085.0,884675.0,b'Router',b'Switch1',39.627970,39.627970,39.637973,0.0,328.064183,18043.5,55.0,0.008437,0.0,39.617967,39.647976,1.030058,50.060098,b'Normal'
4,24.80,8.00,157335.0,23.0,21.0,b'ack',55.0,b'-------',9.0,4901.0,16088.0,884840.0,b'Router',b'Switch1',16.039806,16.039806,16.049810,0.0,328.113525,18046.2,55.0,0.008438,0.0,16.029803,16.059813,1.030054,50.061864,b'Normal'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2160663,24.10,10.00,43184.0,22.0,10.0,b'ack',55.0,b'-------',11.0,1343.0,16103.0,885665.0,b'Switch2',b'clien-10',5.295040,5.295040,5.305043,0.0,328.561918,18070.9,55.0,0.008446,0.0,5.275034,5.305043,1.030013,50.040562,b'Normal'
2160664,24.12,12.00,376386.0,23.0,22.0,b'ack',55.0,b'-------',13.0,10991.0,16103.0,885665.0,b'Router',b'Switch2',34.508649,34.508649,34.518652,0.0,328.491596,18067.0,55.0,0.008446,0.0,34.498646,34.528655,1.030026,50.051067,b'Normal'
2160665,1.00,24.10,140121.0,21.0,23.0,b'tcp',1540.0,b'-------',2.0,4374.0,16091.0,24780100.0,b'Switch1',b'Router',14.418011,14.418011,14.428134,0.0,328.297855,505578.0,1540.0,0.236321,0.0,14.407888,14.438258,1.000000,50.013418,b'Normal'
2160666,5.00,24.50,26800.0,5.0,21.0,b'tcp',1540.0,b'-------',6.0,832.0,16091.0,24780100.0,b'clien-5',b'Switch1',3.732227,3.732227,3.742350,0.0,328.204637,505434.0,1540.0,0.236321,0.0,3.732227,3.762597,1.000000,50.027339,b'Normal'


In [37]:
#decode byte strings to remove the "b" before the srting values at cols PKT_CLASS, NODE_NAME_FROM, NODE_NAME_TO, FLAGS, PKT_TYPE 
categoricals = ["NODE_NAME_FROM", "NODE_NAME_TO", "FLAGS", "PKT_TYPE", "PKT_CLASS"]

for categorical in categoricals:
  df[categorical] = df[categorical].apply(lambda x: x.decode('utf-8')).astype("category") 

df

,SRC_ADD,DES_ADD,PKT_ID,FROM_NODE,TO_NODE,PKT_TYPE,PKT_SIZE,FLAGS,FID,SEQ_NUMBER,NUMBER_OF_PKT,NUMBER_OF_BYTE,NODE_NAME_FROM,NODE_NAME_TO,PKT_IN,PKT_OUT,PKT_R,PKT_DELAY_NODE,PKT_RATE,BYTE_RATE,PKT_AVG_SIZE,UTILIZATION,PKT_DELAY,PKT_SEND_TIME,PKT_RESEVED_TIME,FIRST_PKT_SENT,LAST_PKT_RESEVED,PKT_CLASS
0,3.00,24.30,389693.0,21.0,23.0,tcp,1540.0,-------,4.0,11339.0,16091.0,24780100.0,Switch1,Router,35.529786,35.529786,35.539909,0.0,328.240918,505490.0,1540.0,0.236321,0.0,35.519662,35.550032,1.000000,50.021920,Normal
1,15.00,24.15,201196.0,23.0,24.0,tcp,1540.0,-------,16.0,6274.0,16092.0,24781700.0,Router,server1,20.176725,20.176725,20.186848,0.0,328.205808,505437.0,1540.0,0.236337,0.0,20.156478,20.186848,1.000000,50.030211,Normal
2,24.15,15.00,61905.0,23.0,22.0,ack,55.0,-------,16.0,1930.0,16092.0,885060.0,Router,Switch2,7.049955,7.049955,7.059958,0.0,328.206042,18051.3,55.0,0.008441,0.0,7.039952,7.069962,1.030045,50.060221,UDP-Flood
3,24.90,9.00,443135.0,23.0,21.0,ack,55.0,-------,10.0,12670.0,16085.0,884675.0,Router,Switch1,39.627970,39.627970,39.637973,0.0,328.064183,18043.5,55.0,0.008437,0.0,39.617967,39.647976,1.030058,50.060098,Normal
4,24.80,8.00,157335.0,23.0,21.0,ack,55.0,-------,9.0,4901.0,16088.0,884840.0,Router,Switch1,16.039806,16.039806,16.049810,0.0,328.113525,18046.2,55.0,0.008438,0.0,16.029803,16.059813,1.030054,50.061864,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2160663,24.10,10.00,43184.0,22.0,10.0,ack,55.0,-------,11.0,1343.0,16103.0,885665.0,Switch2,clien-10,5.295040,5.295040,5.305043,0.0,328.561918,18070.9,55.0,0.008446,0.0,5.275034,5.305043,1.030013,50.040562,Normal
2160664,24.12,12.00,376386.0,23.0,22.0,ack,55.0,-------,13.0,10991.0,16103.0,885665.0,Router,Switch2,34.508649,34.508649,34.518652,0.0,328.491596,18067.0,55.0,0.008446,0.0,34.498646,34.528655,1.030026,50.051067,Normal
2160665,1.00,24.10,140121.0,21.0,23.0,tcp,1540.0,-------,2.0,4374.0,16091.0,24780100.0,Switch1,Router,14.418011,14.418011,14.428134,0.0,328.297855,505578.0,1540.0,0.236321,0.0,14.407888,14.438258,1.000000,50.013418,Normal
2160666,5.00,24.50,26800.0,5.0,21.0,tcp,1540.0,-------,6.0,832.0,16091.0,24780100.0,clien-5,Switch1,3.732227,3.732227,3.742350,0.0,328.204637,505434.0,1540.0,0.236321,0.0,3.732227,3.762597,1.000000,50.027339,Normal


In [38]:
df.describe()
df.isnull().values.any()

False

In [40]:
intgs = ["SRC_ADD", "DES_ADD", "PKT_ID", "FROM_NODE", "TO_NODE", "FID"]
for intg in intgs:
  df2[intg] = df[intg].astype('int64')


In [41]:
df.dtypes


SRC_ADD                int64
DES_ADD                int64
PKT_ID                 int64
FROM_NODE              int64
TO_NODE                int64
PKT_TYPE            category
PKT_SIZE             float64
FLAGS               category
FID                    int64
SEQ_NUMBER           float64
NUMBER_OF_PKT        float64
NUMBER_OF_BYTE       float64
NODE_NAME_FROM      category
NODE_NAME_TO        category
PKT_IN               float64
PKT_OUT              float64
PKT_R                float64
PKT_DELAY_NODE       float64
PKT_RATE             float64
BYTE_RATE            float64
PKT_AVG_SIZE         float64
UTILIZATION          float64
PKT_DELAY            float64
PKT_SEND_TIME        float64
PKT_RESEVED_TIME     float64
FIRST_PKT_SENT       float64
LAST_PKT_RESEVED     float64
PKT_CLASS           category
dtype: object

copying the df to one with target dummy variable (y), leaving original intact 

In [51]:
df2 = df.copy()

X = df2.iloc[:, 0:-1]
y = df2.iloc[:, [-1]]

In [52]:
X

,SRC_ADD,DES_ADD,PKT_ID,FROM_NODE,TO_NODE,PKT_TYPE,PKT_SIZE,FLAGS,FID,SEQ_NUMBER,NUMBER_OF_PKT,NUMBER_OF_BYTE,NODE_NAME_FROM,NODE_NAME_TO,PKT_IN,PKT_OUT,PKT_R,PKT_DELAY_NODE,PKT_RATE,BYTE_RATE,PKT_AVG_SIZE,UTILIZATION,PKT_DELAY,PKT_SEND_TIME,PKT_RESEVED_TIME,FIRST_PKT_SENT,LAST_PKT_RESEVED
0,3,24,389693,21,23,tcp,1540.0,-------,4,11339.0,16091.0,24780100.0,Switch1,Router,35.529786,35.529786,35.539909,0.0,328.240918,505490.0,1540.0,0.236321,0.0,35.519662,35.550032,1.000000,50.021920
1,15,24,201196,23,24,tcp,1540.0,-------,16,6274.0,16092.0,24781700.0,Router,server1,20.176725,20.176725,20.186848,0.0,328.205808,505437.0,1540.0,0.236337,0.0,20.156478,20.186848,1.000000,50.030211
2,24,15,61905,23,22,ack,55.0,-------,16,1930.0,16092.0,885060.0,Router,Switch2,7.049955,7.049955,7.059958,0.0,328.206042,18051.3,55.0,0.008441,0.0,7.039952,7.069962,1.030045,50.060221
3,24,9,443135,23,21,ack,55.0,-------,10,12670.0,16085.0,884675.0,Router,Switch1,39.627970,39.627970,39.637973,0.0,328.064183,18043.5,55.0,0.008437,0.0,39.617967,39.647976,1.030058,50.060098
4,24,8,157335,23,21,ack,55.0,-------,9,4901.0,16088.0,884840.0,Router,Switch1,16.039806,16.039806,16.049810,0.0,328.113525,18046.2,55.0,0.008438,0.0,16.029803,16.059813,1.030054,50.061864
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2160663,24,10,43184,22,10,ack,55.0,-------,11,1343.0,16103.0,885665.0,Switch2,clien-10,5.295040,5.295040,5.305043,0.0,328.561918,18070.9,55.0,0.008446,0.0,5.275034,5.305043,1.030013,50.040562
2160664,24,12,376386,23,22,ack,55.0,-------,13,10991.0,16103.0,885665.0,Router,Switch2,34.508649,34.508649,34.518652,0.0,328.491596,18067.0,55.0,0.008446,0.0,34.498646,34.528655,1.030026,50.051067
2160665,1,24,140121,21,23,tcp,1540.0,-------,2,4374.0,16091.0,24780100.0,Switch1,Router,14.418011,14.418011,14.428134,0.0,328.297855,505578.0,1540.0,0.236321,0.0,14.407888,14.438258,1.000000,50.013418
2160666,5,24,26800,5,21,tcp,1540.0,-------,6,832.0,16091.0,24780100.0,clien-5,Switch1,3.732227,3.732227,3.742350,0.0,328.204637,505434.0,1540.0,0.236321,0.0,3.732227,3.762597,1.000000,50.027339


In [53]:
y

,PKT_CLASS
0,Normal
1,Normal
2,UDP-Flood
3,Normal
4,Normal
...,...
2160663,Normal
2160664,Normal
2160665,Normal
2160666,Normal


In [63]:
#split into test and training sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0)

#dummy variables w/ WoE
# polynomial encoder wrapper for selected categorical variables
enc = PolynomialWrapper(ce.WOEEncoder(cols=["NODE_NAME_FROM", "NODE_NAME_TO", "FLAGS", "PKT_TYPE"]))

# transform the datasets
training_numeric_df2 = enc.fit_transform(X_train, y_train)
testing_numeric_df2 = enc.transform(X_test)

training_numeric_df2.info()

/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1512467 entries, 303819 to 305711
Data columns (total 39 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   SRC_ADD                    1512467 non-null  int64  
 1   DES_ADD                    1512467 non-null  int64  
 2   PKT_ID                     1512467 non-null  int64  
 3   FROM_NODE                  1512467 non-null  int64  
 4   TO_NODE                    1512467 non-null  int64  
 5   PKT_SIZE                   1512467 non-null  float64
 6   FID                        1512467 non-null  int64  
 7   SEQ_NUMBER                 1512467 non-null  float64
 8   NUMBER_OF_PKT              1512467 non-null  float64
 9   NUMBER_OF_BYTE             1512467 non-null  float64
 10  PKT_IN                     1512467 non-null  float64
 11  PKT_OUT                    1512467 non-null  float64
 12  PKT_R                      1512467 non-null  float64
 13  PKT_DELA

In [66]:
X_train_enc = training_numeric_df2
X_test_enc = testing_numeric_df2

X_test_enc

,SRC_ADD,DES_ADD,PKT_ID,FROM_NODE,TO_NODE,PKT_SIZE,FID,SEQ_NUMBER,NUMBER_OF_PKT,NUMBER_OF_BYTE,PKT_IN,PKT_OUT,PKT_R,PKT_DELAY_NODE,PKT_RATE,BYTE_RATE,PKT_AVG_SIZE,UTILIZATION,PKT_DELAY,PKT_SEND_TIME,PKT_RESEVED_TIME,FIRST_PKT_SENT,LAST_PKT_RESEVED,NODE_NAME_FROM_UDP-Flood,NODE_NAME_TO_UDP-Flood,FLAGS_UDP-Flood,PKT_TYPE_UDP-Flood,NODE_NAME_FROM_Smurf,NODE_NAME_TO_Smurf,FLAGS_Smurf,PKT_TYPE_Smurf,NODE_NAME_FROM_HTTP-FLOOD,NODE_NAME_TO_HTTP-FLOOD,FLAGS_HTTP-FLOOD,PKT_TYPE_HTTP-FLOOD,NODE_NAME_FROM_SIDDOS,NODE_NAME_TO_SIDDOS,FLAGS_SIDDOS,PKT_TYPE_SIDDOS
2002322,24,8,540367,24,23,55.0,9,15111.0,16088.0,884840.0,47.077610,47.077610,47.087613,0.000000,328.113525,18046.2,55.0,0.008438,0.000000,47.077610,47.107619,1.030054,50.061864,-2.328365,-2.299681,0.00033,-2.293113,-0.334896,-0.368218,0.000014,-0.341387,-5.833252,-6.735180,0.000002,-6.940528,-2.842957,-2.884689,-0.000071,-3.015949
2136657,10,24,120195,10,22,1540.0,11,3756.0,16103.0,24798600.0,12.533669,12.533669,12.543792,0.000000,328.561898,505985.0,1540.0,0.236498,0.000000,12.533669,12.564038,1.000000,50.010552,-2.280913,-2.235052,0.00033,-2.338896,-0.247114,-0.393167,0.000014,-0.225879,-3.377789,-5.817382,0.000002,0.986584,-3.190966,-2.857861,-0.000071,0.962001
238378,11,24,360853,23,24,1540.0,12,10616.0,16103.0,24798600.0,33.326946,33.326946,33.337070,0.000000,328.522879,505925.0,1540.0,0.236498,0.000000,33.306700,33.337070,1.000000,50.016373,-2.294750,0.497294,0.00033,-2.338896,-0.350929,0.110992,0.000014,-0.225879,-6.734427,0.432191,0.000002,0.986584,-2.871194,-2.906790,-0.000071,0.962001
2018992,11,24,57188,11,22,1500.0,32,2192.0,9108.0,13662000.0,54.394286,54.394526,54.404646,0.000240,1016.449678,1524670.0,1500.0,0.130291,0.033360,54.394286,54.427646,1.000000,9.960601,5.432753,4.638322,0.00033,1.580323,1.199246,1.300185,0.000014,-0.732186,2.489721,2.413003,0.000002,-6.594262,-1.950601,-4.264663,-0.000071,-3.386976
1607715,2,24,190839,23,24,1540.0,3,5955.0,16091.0,24780100.0,19.201541,19.201541,19.211664,0.000000,328.264040,505526.0,1540.0,0.236321,0.000000,19.181294,19.211664,1.000000,50.018467,-2.294750,0.497294,0.00033,-2.338896,-0.350929,0.110992,0.000014,-0.225879,-6.734427,0.432191,0.000002,0.986584,-2.871194,-2.906790,-0.000071,0.962001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1007230,24,4,205992,23,21,55.0,5,6419.0,16091.0,885005.0,20.617658,20.617658,20.627661,0.000000,328.218006,18052.0,55.0,0.008440,0.000000,20.607654,20.637664,1.030035,50.055377,-2.294750,-2.312112,0.00033,-2.293113,-0.350929,-0.324046,0.000014,-0.341387,-6.734427,-6.227034,0.000002,-6.940528,-2.871194,-2.841425,-0.000071,-3.015949
1876974,12,24,296879,23,24,1000.0,33,425.0,6250.0,6250000.0,28.420344,28.420627,28.430707,0.000283,124.943426,124943.0,1000.0,0.059605,0.000467,28.400000,28.430707,25.000000,75.022640,-2.294750,0.497294,0.00033,1.580323,-0.350929,0.110992,0.000014,-0.732186,-6.734427,0.432191,0.000002,-6.594262,-2.871194,-2.906790,-0.000071,-3.386976
1857490,0,24,345852,21,23,1540.0,1,10233.0,16091.0,24780100.0,32.165716,32.165716,32.175839,0.000000,328.316229,505606.0,1540.0,0.236321,0.000000,32.155593,32.185962,1.000000,50.010675,-2.291238,-2.299681,0.00033,-2.338896,-0.376882,-0.368218,0.000014,-0.225879,-6.226319,-6.735180,0.000002,0.986584,-3.018988,-2.884689,-0.000071,0.962001
642338,24,6,117524,24,23,55.0,7,3655.0,16091.0,885005.0,12.279883,12.279883,12.289886,0.000000,328.193090,18050.6,55.0,0.008440,0.000000,12.279883,12.309893,1.030048,50.059112,-2.328365,-2.299681,0.00033,-2.293113,-0.334896,-0.368218,0.000014,-0.341387,-5.833252,-6.735180,0.000002,-6.940528,-2.842957,-2.884689,-0.000071,-3.015949


In [69]:
#TODO normalize

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_enc)
X_test_scaled = scaler.fit_transform(X_test_enc)

In [70]:
X_test_scaled

array([[ 1.11952814, -1.19607929,  1.27748827, ..., -0.24040455,
        -0.019877  , -0.6964335 ],
       [-0.49851185,  0.71197053, -0.87051728, ..., -0.22056381,
        -0.019877  ,  1.28836133],
       [-0.38293756,  0.71197053,  0.35977584, ..., -0.25674982,
        -0.019877  ,  1.28836133],
       ...,
       [-1.6542547 ,  0.71197053,  0.28308765, ..., -0.24040455,
        -0.019877  ,  1.28836133],
       [ 1.11952814, -1.43458551, -0.88417198, ..., -0.24040455,
        -0.019877  , -0.6964335 ],
       [-1.6542547 ,  0.71197053, -0.64767624, ..., -0.25674982,
        -0.019877  ,  1.28836133]])

The “lbfgs”, “sag” and “newton-cg” solvers only support 
 regularization or no regularization, and are found to converge faster for some high-dimensional data. Setting multi_class to “multinomial” with these solvers learns a true multinomial logistic regression model, which means that its probability estimates should be better calibrated than the default “one-vs-rest” setting.

The “sag” solver uses Stochastic Average Gradient descent (see https://hal.inria.fr/hal-00860051/document). It is faster than other solvers for large datasets, when both the number of samples and the number of features are large.

In [73]:
#instantiate the model - multinomial w/ sag solver #TODO: fix, model is not converging
'''
TODO: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
array([[  1207,      0,     75,      0,      0],
       [    13, 581075,     60,      0,      0],
       [     0,    117,   1860,      0,      0],
       [     9,   2490,     88,   1222,      0],
       [     0,   5939,      0,      0,  54046]]) 
'''

logreg = LogisticRegression(solver='sag', C=0.05, multi_class='multinomial',
                          random_state=42)

# fit the model with data
logreg.fit(X_train_scaled, y_train)

#
y_pred=logreg.predict(X_test_scaled)

# import the metrics class
from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


array([[  1207,      0,     75,      0,      0],
       [    13, 581075,     60,      0,      0],
       [     0,    117,   1860,      0,      0],
       [     9,   2490,     88,   1222,      0],
       [     0,   5939,      0,      0,  54046]])